In [1]:
# 필요한 라이브러리 임포트&설치
# !pip install dbfread
# !pip install haversine
# !pip install sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

In [2]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


In [3]:
gu_list = ['종로구']

In [3]:
gu_list = ['종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구']
gu_list

['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구']

# 첫페이지, 기사제목 뽑기

In [5]:
import requests
from bs4 import BeautifulSoup
for keyword in gu_list:

#     keyword
    url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}%2B초등+%2B돌봄+&oquery={keyword}'      

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    news_titles = soup.select('.news .type01 li dt a[title]')

#     print('총', len(news_titles), '개의 뉴스 제목이 있습니다')
    print()
    for title in news_titles:
        print(title['title'])


노래방·PC방·커피숍·교습소에 100~200만원... 초등생 이하 1인당 20만원
'PC방 아빠·방판 엄마·취준생 딸·초등아들…426만원 받는다
‘발등의 불’ 초등돌봄… 서울시, 맞벌이 부부 긴급 서비스 시행
서울 강서 유치원 4곳·초등 7곳 '등교 연기'…양천 2곳은 6월 등교
[일문일답] 유은혜 "27일 등교로 초등 돌봄 없어지지 않아"
등교 하루 앞두고 유·초등 연기 잇따라...서울교육청, 등교 후속대책 발표
등교 하루 앞두고 서울 초등·유치원 16곳 내달로 등교연기(종합2보)
등교하는 긴급돌봄 참여 초등학생
수도권 요양·정신병원 전수검사…등교 수업 확대
등교 앞둔 초등·유치원생 학부모들 '어쩌나'

류규하 대구 중구청장 "사람과 마을 잇는 3단계 여성친화도시 도전"
성동구 "타 자치구 확진자, 관내 동호초등학교 방문…269명 검사"
중구 초등 비만예방 프로그램 '건강한 돌봄놀이터' 운영
부산 중구 전국 지방자치단체 일자리대상 우수사업부문 우수상 수상 外
초등 돌봄교실 운영이 지자체 몫이어야 하는 까닭
서울 중구 직영 '초등 돌봄교실'…학부모 "만족도 99%"
서울시·교육청 '촘촘한 초등돌봄' 힘 모은다
오후 8시까지 초등생 봐주는 ‘중구돌봄’ 만족도 99%
서양호 중구청장 취임 2주년 첫날부터 현장 행정
중구 초등돌봄교사 '특별휴가'

[현장] 초등 저학년도 온라인개학…초중고 540만 명 원격수업
용산구, 우리동네 키움센터 5호점 문 열어
서울 용산구, 아동돌봄공간·청년창업센터 등 구민시설 완공
서울시교육청, 8일 관내 공립초등학교 예비소집 실시
서울 용산구, 새로운 개념의 구민편의 복합시설 조성
개학 연기 초등학교...돌봄교실은 희망자 한해서 운영
'학원강사발 감염' 인천, 내주 27일 초등생 등교 가능하려나
학부모 10명 중 8명 "초등돌봄교실 법제화 찬성"
"2024년부터 초등학교 하교시간 연장"…전문가들 난상토론
화재대피훈련 하는 원효초등학교 학생들

성동구 "타 자치구 확진자, 관내 동호초등학교 방문…269명 검사"
지친 초등생에… 성동구, 

# 제목, 날짜, 신문사, url

In [4]:
from bs4 import BeautifulSoup
import requests
import csv


for keyword in tqdm(gu_list):

#     keyword
#     url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}%2B초등+%2B돌봄+&oquery={keyword}'      

    csv_filename = "naver_news_{}.csv".format(keyword)
    csv_open = open(csv_filename, "w+", encoding='utf-8')
    csv_writer = csv.writer(csv_open)
    csv_writer.writerow( ('제목', '날짜', '신문사', '링크', ) )  #제목 입력

    # 총 100까지, 10페이지간의 간격으로 진행
    for n in range(1,100,10):
        req = requests.get("https://search.naver.com/search.naver?&where=news&query={}%2B초등+%2B돌봄+&oquery={}&start=".format(keyword,keyword)+str(n), 
                            headers={'User-Agent':'Mozilla/5.0'})
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        articles = soup.select('ul.type01 > li')

        for article in articles:
            title = article.select_one("a._sp_each_title").text          
            # 기사 제목
            others = article.select_one("dd.txt_inline").text          
            # date 추출을 위해 여러 정보를 불러옴
            agency = article.select_one("span._sp_each_source").text     
            # 언론사
            link = article.select_one("a._sp_each_url")['href']

            agency = agency.rstrip("언론사 선정")                           
            # 불필요한 문자 삭제

            start = others.find('2')
            end = start + 10
            article_date = others[start:end]

            csv_writer.writerow ( (title, article_date, agency, link, ) )

    csv_open.close()

#     pd.read_csv('{}'.format(keyword), encoding='utf-8')

In [9]:
for keyword in gu_list:
    print(pd.read_csv('naver_news_{}.csv'.format(keyword), encoding='utf-8'))

                                               제목          날짜     신문사  \
0   노래방·PC방·커피숍·교습소에 100~200만원... 초등생 이하 1인당 20만원  2020.09.10   오마이뉴스   
1              'PC방 아빠·방판 엄마·취준생 딸·초등아들…426만원 받는다  2020.09.14    이데일리   
2             ‘발등의 불’ 초등돌봄… 서울시, 맞벌이 부부 긴급 서비스 시행  2020.08.27    국민일보   
3         서울 강서 유치원 4곳·초등 7곳 '등교 연기'…양천 2곳은 6월 등교  2020.05.26  한국경제TV   
4              [일문일답] 유은혜 "27일 등교로 초등 돌봄 없어지지 않아"  2020.05.24     뉴스1   
..                                            ...         ...     ...   
95                          미니학교를 인기학교로 만든 학부모의 힘  2011.10.27    연합뉴스   
96                         종로소방서, 창신초 돌봄교실 소방안전교육  2019.12.30     FPN   
97            [MT리포트]끝이 안보이는 '에듀푸어'…교육도 결국은 '돈'문제  2018.06.14   머니투데이   
98                       통폐합 대신 작은학교 살리기… 조희연의 실험  2016.10.13    서울신문   
99                         종로소방서, 창신초 돌봄교실 소방안전교육  2019.07.30     FPN   

                                                   링크  
0   http://www.ohmynews.com/NWS_Web/View/at_pg.asp...  
1   http://

In [10]:
pd.read_csv('naver_news_종로구.csv', encoding='utf-8')

,제목,날짜,신문사,링크
0,노래방·PC방·커피숍·교습소에 100~200만원... 초등생 이하 1인당 20만원,2020.09.10,오마이뉴스,http://www.ohmynews.com/NWS_Web/View/at_pg.asp...
1,'PC방 아빠·방판 엄마·취준생 딸·초등아들…426만원 받는다,2020.09.14,이데일리,http://www.edaily.co.kr/news/newspath.asp?news...
2,"‘발등의 불’ 초등돌봄… 서울시, 맞벌이 부부 긴급 서비스 시행",2020.08.27,국민일보,http://news.kmib.co.kr/article/view.asp?arcid=...
3,서울 강서 유치원 4곳·초등 7곳 '등교 연기'…양천 2곳은 6월 등교,2020.05.26,한국경제TV,http://www.wowtv.co.kr/NewsCenter/News/Read?ar...
4,"[일문일답] 유은혜 ""27일 등교로 초등 돌봄 없어지지 않아""",2020.05.24,뉴스1,https://www.news1.kr/articles/?3943681
...,...,...,...,...
95,미니학교를 인기학교로 만든 학부모의 힘,2011.10.27,연합뉴스,https://news.naver.com/main/read.nhn?mode=LSD&...
96,"종로소방서, 창신초 돌봄교실 소방안전교육",2019.12.30,FPN,http://fpn119.co.kr/128564
97,[MT리포트]끝이 안보이는 '에듀푸어'…교육도 결국은 '돈'문제,2018.06.14,머니투데이,http://news.mt.co.kr/mtview.php?no=20180613140...
98,통폐합 대신 작은학교 살리기… 조희연의 실험,2016.10.13,서울신문,http://www.seoul.co.kr/news/newsView.php?id=20...


# 네이버 기사댓글 크롤링

In [24]:
link_list=[]
for keyword in gu_list:

#     keyword
#     url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}%2B초등+%2B돌봄+&oquery={keyword}'      

#     csv_filename = "naver_news_{}.csv".format(keyword)
#     csv_open = open(csv_filename, "w+", encoding='utf-8')
#     csv_writer = csv.writer(csv_open)
#     csv_writer.writerow( ('제목', '날짜', '신문사', '링크', ) )  #제목 입력

    # 총 100까지, 10페이지간의 간격으로 진행
    for n in range(1,100,10):
        req = requests.get("https://search.naver.com/search.naver?&where=news&query={}%2B초등+%2B돌봄+&oquery={}&start=".format(keyword,keyword)+str(n), 
                            headers={'User-Agent':'Mozilla/5.0'})
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        articles = soup.select('ul.type01 > li')

        for article in articles:
#             title = article.select_one("a._sp_each_title").text          
#             # 기사 제목
#             others = article.select_one("dd.txt_inline").text          
#             # date 추출을 위해 여러 정보를 불러옴
#             agency = article.select_one("span._sp_each_source").text     
#             # 언론사
            link = article.select_one("a._sp_each_title")['href']
            link_list.append(link)
#             agency = agency.rstrip("언론사 선정")                           
#             # 불필요한 문자 삭제

#             start = others.find('2')
#             end = start + 10
#             article_date = others[start:end]

#             csv_writer.writerow ( (title, article_date, agency, link, ) )

#     csv_open.close()

In [30]:
k = pd.DataFrame(link_list)
k

,0
0,http://www.ohmynews.com/NWS_Web/View/at_pg.asp...
1,http://www.edaily.co.kr/news/newspath.asp?news...
2,http://news.kmib.co.kr/article/view.asp?arcid=...
3,http://www.wowtv.co.kr/NewsCenter/News/Read?ar...
4,https://www.news1.kr/articles/?3943681
...,...
895,http://www.youthdaily.co.kr/news/article.html?...
896,http://news.mk.co.kr/newsRead.php?year=2019&no...
897,http://www.segye.com/content/html/2020/03/16/2...
898,http://www.ohmynews.com/NWS_Web/View/at_pg.asp...


In [31]:
k.to_pickle('link_list.pkl')

In [7]:
kk = pd.read_pickle('link_list.pkl')
kk

,0
0,http://www.ohmynews.com/NWS_Web/View/at_pg.asp...
1,http://www.edaily.co.kr/news/newspath.asp?news...
2,http://news.kmib.co.kr/article/view.asp?arcid=...
3,http://www.wowtv.co.kr/NewsCenter/News/Read?ar...
4,https://www.news1.kr/articles/?3943681
...,...
895,http://www.youthdaily.co.kr/news/article.html?...
896,http://news.mk.co.kr/newsRead.php?year=2019&no...
897,http://www.segye.com/content/html/2020/03/16/2...
898,http://www.ohmynews.com/NWS_Web/View/at_pg.asp...


# 무슨박사 네이버뉴스포멧 본문크롤링

In [4]:
import requests
from bs4 import BeautifulSoup
import json
import re
import sys
import time, random

# query={}%2B초등+%2B돌봄+&oquery={}
def get_news(n_url):
    news_detail = []
    print(n_url)
    breq = requests.get(n_url)
    bsoup = BeautifulSoup(breq.content, 'html.parser')

    # html 파싱
    title = bsoup.select('h3#articleTitle')[0].text
    news_detail.append(title)

    # 날짜 파싱
    pdate = bsoup.select('.t11')[0].get_text()[:11]
    news_detail.append(pdate)

    # 기사 본문 크롤링 
    _text = bsoup.select('#articleBodyContents')[0].get_text().replace('\n', " ")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())

    # 신문사 크롤링
    pcompany = bsoup.select('#footer address')[0].a.get_text()
    news_detail.append(pcompany)

    return news_detail


# 쿼리에 검색어를 입력하고 검색 시작날짜부터 끝 날짜까지를 입력합니다.
query = "광진구"   # url 인코딩 에러는 encoding parse.quote(query)

s_date = "2020.01.01"
e_date = "2020.10.14"
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")
page = 1

# 저장 경로를 입력합니다.
f = open("./" + query + 'test.txt', 'w', encoding='utf-8')
links = []
# 최대 몇개까지 크롤링을 할지 숫자를 입력합니다. 크롤링 순서는 최신기사 순서로 크롤링 됩니다.
while page < 100:
   
    print(page)
    
    url = "https://search.naver.com/search.naver?where=news&query="+query+"%2B초등+%2B돌봄+&oquery="+query+"&sort=1&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
    #header 추가
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    req = requests.get(url,headers=header)
    print(url)
#     links.append(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
        #print(soup)
    
    for urls in soup.select("._sp_each_url"):
        try :
            #print(urls["href"])
            if urls["href"].startswith("https://news.naver.com"):
                #print(urls["href"])
                news_detail = get_news(urls["href"])
                    # pdate, pcompany, title, btext
                f.write("{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[3], news_detail[0],
                                                      news_detail[2]))  # new style
        except Exception as e:
            print(e) 
            continue
    page += 10  
f.close()

1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

...

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2Cp%3Afrom20200101to20201014%2Ca%3A&start=1
1
https://search.naver.com/search.naver?where=news&query=광진구%2B초등+%2B돌봄+&oquery=광진구&sort=1&ds=2020.01.01&de=2020.10.14&nso=so%3Ar%2C

KeyboardInterrupt: 

In [ ]:
links[0]

In [18]:
from selenium import webdriver
import time
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=025&aid=0003044323'

#웹 드라이버
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(30)
driver.get(url)

#더보기 계속 클릭하기
while True:
    try:
        더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
        더보기.click()
        time.sleep(1)
    except:
        break

In [5]:
from selenium import webdriver
import time
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0003044257'

#웹 드라이버
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(30)
driver.get(url)

#더보기 계속 클릭하기
while True:
    try:
        더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
        더보기.click()
        time.sleep(1)
    except:
        break

#댓글추출
contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
for content in contents:
    print(content.text)

끝까지 코로나 정책을 하겠다는거지.. "징그럽다 문재앙"
배달업체 세금 팍팍메기고, 배달오토바이들이나 신호무시에 인도침범좀 못하게해라..
제발 삽질좀 하지마세요. 삑삑대면서 재난문자 수차례 돌리는 문자내용이 가족간 자차이용, 불가피한외출자제입니다. 수신거부도 못하는 재난문자 그리 보내놓곤, 지난번에도 이런분위기 조성해서 코로나 재확산시켜놓고 지금 소비쿠폰? 방역원칙 지키고, 그것보다 더 높은 수준으로 나를위해, 남을위해, 가족을위해서 외출자제하고있는 사람들은 뭐가됩니까? 게다가 그 지원금도 세금이잖아요? 통신비도 연령별 차등지원한다고하질않나.. 막말로 아들딸생각해서 외출자제하면서, 근면성실하게 돈버는 가장맞벌이들은 나갈시간도 없고 그나이대가 딱통신비2만원도 안됩니다.
응 안봐요 ~ CGV 2천원인상 안봐요집에서 안나와요 문재앙
하지말자. 제발... 이런거
제발 이란짓 좀 하지마세요
아직 코로나 끝나지도 않았는데 확진되면 누구책임으로 몰려고 이런짓을 하나?? 마스크도 필수로 쓰게했으면서 영화관 이딴곳을 갈 시국이냐?? 일부로 코로나를 끌고갈려고하는속셈인거야??
또 쿠폰 뿌려 문화생활 즐겨라해놓고 확산세 돌면 어디에다가 뒤집어씌우려고요 또또또? 문재인퇴진 집회 애국자들한테겠지 교회단체랑 4.15부정선거조작


In [21]:
from selenium import webdriver
import time
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0003044257'

#웹 드라이버
driver = webdriver.Chrome('./chromedriver.exe')
driver.implicitly_wait(30)
driver.get(url)

#더보기 계속 클릭하기
while True:
    try:
        더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
        더보기.click()
        time.sleep(1)
    except:
        break

#댓글추출
contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
# for content in contents:
#     print(content.text)
contents = [content.text for content in contents]

#작성자
nicks = driver.find_elements_by_css_selector('span.u_cbox_nick')
# for nick in nicks:
#     print(nick.text)
nicks = [nick.text for nick in nicks]

#날짜 추출
dates = driver.find_elements_by_css_selector('span.u_cbox_date')
# for date in dates:
#     print(date.text)
dates = [date.text for date in dates]

# 취합
replys = list(zip(nicks,dates,contents))

driver.quit()

In [ ]:
from selenium import webdriver
import time

def get_replys(url,imp_time=5,delay_time=0.1):

    #웹 드라이버
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.implicitly_wait(imp_time)
    driver.get(url)

    #더보기 계속 클릭하기
    while True:
        try:
            더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            더보기.click()
            time.sleep(delay_time)
        except:
            break

    #댓글추출
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    # for content in contents:
    #     print(content.text)
    contents = [content.text for content in contents]

    #작성자
    nicks = driver.find_elements_by_css_selector('span.u_cbox_nick')
    # for nick in nicks:
    #     print(nick.text)
    nicks = [nick.text for nick in nicks]

    #날짜 추출
    dates = driver.find_elements_by_css_selector('span.u_cbox_date')
    # for date in dates:
    #     print(date.text)
    dates = [date.text for date in dates]

    # 취합
    replys = list(zip(nicks,dates,contents))

    driver.quit()
    return replys

if __name__ == '__main__':
    from datetime import datetime
    start = datetime.now()

    url = 'https://entertain.naver.com/ranking/comment/list?oid=022&aid=0003409199'
    reply_data = get_replys(url, 5, 0.1)

In [ ]:
from selenium import webdriver
import time

def get_replys(url,imp_time=5,delay_time=0.1):

    #웹 드라이버
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.implicitly_wait(imp_time)
    driver.get(url)

    #더보기 계속 클릭하기
    while True:
        try:
            더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            더보기.click()
            time.sleep(delay_time)
        except:
            break

    #댓글추출
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    # for content in contents:
    #     print(content.text)
    contents = [content.text for content in contents]

    #작성자
    nicks = driver.find_elements_by_css_selector('span.u_cbox_nick')
    # for nick in nicks:
    #     print(nick.text)
    nicks = [nick.text for nick in nicks]

    #날짜 추출
    dates = driver.find_elements_by_css_selector('span.u_cbox_date')
    # for date in dates:
    #     print(date.text)
    dates = [date.text for date in dates]

    # 취합
    replys = list(zip(nicks,dates,contents))

    driver.quit()
    return replys

if __name__ == '__main__':
    url = 'https://entertain.naver.com/ranking/comment/list?oid=022&aid=0003409199'
    reply_data = get_replys(url,5,0.1)

    import pandas as pd #pandas, openpyxl
    col =[ '작성자','날짜','내용']
    data_frame = pd.DataFrame(reply_data,columns=col)
    data_frame.to_excel('news.xlsx',sheet_name='방탄소년단 지민',startrow=0,header=True)


In [ ]:
from selenium import webdriver
import time

def get_replys(url,imp_time=5,delay_time=0.1):

    #웹 드라이버
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.implicitly_wait(imp_time)
    driver.get(url)

    #더보기 계속 클릭하기
    while True:
        try:
            더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            더보기.click()
            time.sleep(delay_time)
        except:
            break

    #댓글추출
    contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    # for content in contents:
    #     print(content.text)
    contents = [content.text for content in contents]

    #작성자
    nicks = driver.find_elements_by_css_selector('span.u_cbox_nick')
    # for nick in nicks:
    #     print(nick.text)
    nicks = [nick.text for nick in nicks]

    #날짜 추출
    dates = driver.find_elements_by_css_selector('span.u_cbox_date')
    # for date in dates:
    #     print(date.text)
    dates = [date.text for date in dates]

    # 취합
    replys = list(zip(nicks,dates,contents))

    driver.quit()
    return replys

if __name__ == '__main__':
    from datetime import datetime
    start = datetime.now()

    url = 'https://entertain.naver.com/ranking/comment/list?oid=022&aid=0003409199'
    reply_data = get_replys(url,5,0.1)

    import pandas as pd #pandas, openpyxl
    col =[ '작성자','날짜','내용']
    data_frame = pd.DataFrame(reply_data,columns=col)
    data_frame.to_excel('news.xlsx',sheet_name='방탄소년단 지민',startrow=0,header=True)

    end = datetime.now()
    print(end-start)

In [ ]:
빠르다는거

In [22]:
from selenium import webdriver
import time

def get_replys(url,imp_time=5,delay_time=0.1):

    #웹 드라이버
    driver = webdriver.Chrome('./chromedriver.exe')
    driver.implicitly_wait(imp_time)
    driver.get(url)

    #더보기 계속 클릭하기
    while True:
        try:
            더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
            더보기.click()
            time.sleep(delay_time)
        except:
            break

    html = driver.page_source
    # print(html)

    # 모듈 참조
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(html, 'lxml') #html.parser

    # 댓글추출
    contents = soup.select('span.u_cbox_contents')
    contents = [content.text for content in contents]

    #작성자
    nicks = soup.select('span.u_cbox_nick')
    nicks = [nick.text for nick in nicks]

    #날짜 추출
    dates = soup.select('span.u_cbox_date')
    dates = [date.text for date in dates]

    # 취합
    replys = list(zip(nicks,dates,contents))

    driver.quit()
    return replys

if __name__ == '__main__':
    from datetime import datetime
    start = datetime.now()

    url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0003044257'
    # url = 'https://entertain.naver.com/ranking/comment/list?oid=076&aid=0003490754'
    reply_data = get_replys(url,5,0.1)

    import pandas as pd #pandas, openpyxl
    col =[ '작성자','날짜','내용']
    data_frame = pd.DataFrame(reply_data,columns=col)
    data_frame.to_excel('news.xlsx',sheet_name='신서유기',startrow=0,header=True)

    end = datetime.now()
    print(end-start)

0:00:11.528629


# url뽑기+하은댓글뽑기

In [9]:
from bs4 import BeautifulSoup
import requests
import csv


for keyword in tqdm(gu_list):

#     keyword
#     url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}%2B초등+%2B돌봄+&oquery={keyword}'      

    csv_filename = "naver_news_{}.csv".format(keyword)
    csv_open = open(csv_filename, "w+", encoding='utf-8')
    csv_writer = csv.writer(csv_open)
    csv_writer.writerow( ('제목', '날짜', '신문사', '링크', ) )  #제목 입력

    # 총 100까지, 10페이지간의 간격으로 진행
    for n in range(1,100,10):
        req = requests.get("https://search.naver.com/search.naver?&where=news&query={}%2B초등+%2B돌봄+&oquery={}&start=".format(keyword,keyword)+str(n), 
                            headers={'User-Agent':'Mozilla/5.0'})
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        articles = soup.select('ul.type01 > li')

        for article in articles:
            title = article.select_one("a._sp_each_title").text          
            # 기사 제목
            others = article.select_one("dd.txt_inline").text          
            # date 추출을 위해 여러 정보를 불러옴
            agency = article.select_one("span._sp_each_source").text     
            # 언론사
            link = article.select_one("a._sp_each_url")['href']

            agency = agency.rstrip("언론사 선정")                           
            # 불필요한 문자 삭제

            start = others.find('2')
            end = start + 10
            article_date = others[start:end]
            print(link)
#             csv_writer.writerow ( (title, article_date, agency, link, ) )

#     csv_open.close()

# #     pd.read_csv('{}'.format(keyword), encoding='utf-8')

PermissionError: [Errno 13] Permission denied: 'naver_news_종로구.csv'

In [12]:

for i in gu_list:
    url_num = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query={}+%EC%B4%88%EB%93%B1%EB%8F%8C%EB%B4%84&oquery=%EB%8F%84%EB%B4%89%EA%B5%AC&tqi=UG0DidprvN8ssiru4ZCssssst28-268643'.format(i)
    resp = requests.get(url_num)
    soup = BeautifulSoup(resp.content, 'lxml')
    num = soup.select('div.section_head div span')[0]
    num = re.findall('\/\s(.+?)건', str(num))
    num = int(num[0].replace(",",''))//10 * 10 + 1
    print(num)

1441


In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

a = {}
for keyword in tqdm(gu_list):
    url_num = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query={}+%EC%B4%88%EB%93%B1%EB%8F%8C%EB%B4%84&oquery=%EB%8F%84%EB%B4%89%EA%B5%AC&tqi=UG0DidprvN8ssiru4ZCssssst28-268643'.format(keyword)
    resp = requests.get(url_num)
    soup = BeautifulSoup(resp.content, 'lxml')
    num = soup.select('div.section_head div span')[0]
    num = re.findall('\/\s(.+?)건', str(num))
    num = int(num[0].replace(",",''))//10 * 10 + 1
#     print(num)

#     # 총 100까지, 10페이지간의 간격으로 진행
    for n in range(1,num,10):
        req = requests.get("https://search.naver.com/search.naver?&where=news&query={}%2B초등+%2B돌봄+&oquery={}&start=".format(keyword,keyword)+str(n), 
                            headers={'User-Agent':'Mozilla/5.0'})
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')

        articles = soup.select('ul.type01 > li')

        for article in articles:
            title = article.select_one("a._sp_each_title").text          
            # 기사 제목
            others = article.select_one("dd.txt_inline").text          
            # date 추출을 위해 여러 정보를 불러옴
            agency = article.select_one("span._sp_each_source").text     
            # 언론사
            link = article.select_one("a._sp_each_url")['href']
            #링크
            agency = agency.rstrip("언론사 선정")                           
            # 불필요한 문자 삭제

            start = others.find('2')
            end = start + 10
            article_date = others[start:end]
            a[title] = link 
         
#             List=[]
#             from bs4 import BeautifulSoup
#             import requests
#             import re
#             import sys
#             import pprint


#             url =link

#             oid=url.split("oid=")[1].split("&")[0]
#             aid=url.split("aid=")[1]
#             page=1    
#             header = {
#                 "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
#                 "referer":url,

#             } 
#             while True :
#                 c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE" 
#                 r=requests.get(c_url,headers=header)
#                 cont=BeautifulSoup(r.content,"html.parser")    
#                 total_comm=str(cont).split('comment":')[1].split(",")[0]

#                 match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
#                 List.append(match)
#                 if int(total_comm) <= ((page) * 20):
#                     break
#                 else : 
#                     page+=1


#             def flatten(l):
#                 flatList = []
#                 for elem in l:
#                     if type(elem) == list:
#                         for e in elem:
#                             flatList.append(e)
#                     else:
#                         flatList.append(elem)
#                 return flatList


#             flatten(List)
#             break

In [19]:
len(a)

90

In [17]:
List=[]
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint


url = a[0]

oid=url.split("oid=")[1].split("&")[0]
aid=url.split("aid=")[1]
page=1    
header = {
    "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
    "referer":url,

} 
while True :
    c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE" 
    r=requests.get(c_url,headers=header)
    cont=BeautifulSoup(r.content,"html.parser")    
    total_comm=str(cont).split('comment":')[1].split(",")[0]

    match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
    List.append(match)
    if int(total_comm) <= ((page) * 20):
        break
    else : 
        page+=1


def flatten(l):
    flatList = []
    for elem in l:
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList


flatten(List)


['"전에 7세미만 줬으면 이번엔 빼야지 왜 또 줘요"',
 '"초딩만 돈드나요? 중딩 고딩 대딩은 돈먹는 하마인데.. 줄려면 다 줘요. 편가르기 제발 그만! 짜증나요"',
 '"2만원받고 얼마를 토해내야할지...\\n차라리 안받고 세금이나 안올랐으면..."',
 '"청년 구직은 청년지지율 때문에 하는 것 같고, 통신비 2만원은 폰 안쓰는 사람 없으니 전 국민한테 조금 인심 쓰는척 하는 돈이네. 전자와 후자 둘 다 필요 없는 세금 소비다. 그 돈으로 벤처기업이나 미래 기술에 투자해라. 그게 구직에 더 도움 될거다."',
 '"학원이 집합금지고 교습소는 아니데 어떻게 된거에요?"',
 '"미취학 아이들과 부모가 얼마나 사회와 국가에 크게 이바지를 했는지는 몰라도 이건 아니죠. 원비도 혜택받고 달마다 꼬박꼬박 돈받아 적금을 넣었네 외식을 했네 이런 소리 듣는것도 짜증납니다. 지금  고딩들은 예방접종도 돈다 내고 맞고 유치원비도 돈다 내고 다닌 애들이 부분이고 중학교때 교복도 제돈 다 내고 샀던 애들입니다. 근데 미취학 애들도 주는 재난 지원금을 고딩 아이들 빼고 준다니 참 차별의 서러움보다도 울화가 미칠고 약올라서 반감생기네요 미취학 아이들과 부모들이 뭐 그리 대단하길래 이런 차별을 하는지 정말 궁금하네요"',
 '"7세미만 아동은 지난번 아동볼봄쿠폰 40만원에\\n매달 양육수당도 지원되고 있는데 이번에 또 준다니요? 초등학교 돌봄교실 있지만 이용하는것 제한있고, 중,고생 학교 못가고 있는데.. 그에 비해 어린이집, 유치원 등원을 막은적이있는가? 긴급돌봄이란 명목하에 부모 맞벌이인지 아닌지 초등학교처럼 서류도 안받고 다들 보내지 않았나? 사실상 돌봄공백이 없는 계층에 무분별한 중복지원을 하고 있다."',
 '"이야... 중..고딩은요? 아랫집 중딩 배가 고프다는소리를 엄청 들어서 만날때마다 이것저것 간식 줬는데.. 그아이들은요? 어린이집 긴급이다 뭐다 할때 아이들 욱여 놓은게 엄마들인데요?\\n제 주변만 봐도 엄마들 집에서 뒹글뒹글 놀고 아이들어린이집에 보냈어요.